In [1]:
import sys
sys.path.append("../src")

import warnings
warnings.filterwarnings("ignore")

import json
import pandas as pd
from pathlib import Path
from tqdm.auto import tqdm

In [2]:
from Classes.cognitive_model_agents import MODELS, M1_MODELS, M2_MODELS, M3_MODELS
from Classes.parameter_recovery import ParameterFit

## Fit to 2-player data

In [3]:
# Create paths for data and results
data_folder = Path('..', 'data', 'human')
folder_2P = Path('..', 'reports', 'MLE')
folder_2P.mkdir(parents=True, exist_ok=True)

file_name = f'best_fit_2P_2.json'
best_fit_file_2P = Path(folder_2P, file_name)

In [4]:
#Load 2P data

file_name = '2-player-UR.csv'
file = data_folder / Path(file_name)
print(f'Loading data from {file}...')
data = pd.read_csv(file)
data.head(2)

Loading data from ..\data\human\2-player-UR.csv...


,Unnamed: 0.4,Unnamed: 0.3,Unnamed: 0.2,Unnamed: 0.1,Unnamed: 0,source,treatment,group,round,player,choice,score,attendance,ac_score,threshold,num_players,room,decision
0,0,0,0,0,0,human,score + grid,Grupo-0001,1,405186428721292,0,0,1,25,0.5,2,Grupo-0001,0
1,1,1,1,1,1,human,score + grid,Grupo-0001,1,728608683559397,1,1,1,25,0.5,2,Grupo-0001,1


In [5]:
# check_out_these = ['Attendance-M2', 'MFP-M2', 'Payoff-M2']
check_out_these = ['Fairness-M2']
my_models = [model for model in MODELS if model.name() in check_out_these]

ParameterFit.run(
    data=data,
    model_list=my_models,
    # model_list=M2_MODELS,
    best_fit_path=best_fit_file_2P,
    new_file=False
)

Fitting models...:   0%|          | 0/1 [00:00<?, ?it/s]

Fitting data to model Fairness-M2...
Creating parameter recovery class...
Running bayesian optimizer...
Finding deviance for 2 players and threshold 0.5...
|   iter    |  target   |   bias    | invers... | learni... |
-------------------------------------------------------------
| 1         | -3.172e+0 | 0.2085    | 46.38     | 0.0001144 |
| 2         | -3.249e+0 | 0.1512    | 10.25     | 0.09234   |
| 3         | -7.475e+0 | 0.09313   | 22.77     | 0.3968    |
| 4         | -8.486e+0 | 0.2694    | 27.41     | 0.6852    |
| 5         | -4.472e+0 | 0.1022    | 56.32     | 0.02739   |
| 6         | -8.141e+0 | 0.3352    | 27.29     | 0.5587    |
| 7         | -8.417e+0 | 0.07019   | 13.48     | 0.8007    |
| 8         | -8.136e+0 | 0.4841    | 20.75     | 0.6923    |
| 9         | -3.188e+0 | 0.1883    | 8.763     | 0.0       |
| 10        | -2.86e+03 | 0.1827    | 48.79     | 0.00674   |
| 11        | -8.883e+0 | 0.5       | 51.97     | 1.0       |
| 12        | -8.86e+03 | 0.5       | 

## Fit to all player data

In [ ]:
# Create paths for data and results
folder_all = Path('..', 'reports', 'MLE')
folder_all.mkdir(parents=True, exist_ok=True)

best_fit_file_all = Path(folder_all, f'best_fit_all.json')

In [ ]:
# Load data into a dataframe
file_names = [
	# '2-player-UR.csv',
	'3-player-IU.csv',
	'4-player-IU.csv',
	'5-player-IU.csv',
	'6-player-IU.csv',
	'7-player-IU.csv',
	'8-player-IU.csv',
	'9-player-IU.csv',
	'11-player-IU.csv',
	'12-player-IU.csv',
]
df_list = list()
for file_name in file_names:
	file = data_folder / Path(file_name)
	print(f'Loading data from {file}...')
	df = pd.read_csv(file)
	df_list.append(df)
data = pd.concat(df_list, ignore_index=True)
columns = ['threshold', 'num_players', 'group', 'round', 'player', 'score', 'decision']
drop_columns = [col for col in data.columns if col not in columns]
data.drop(columns=drop_columns, inplace=True)
data.head()

In [ ]:
ParameterFit.run(
    data=data,
    model_list=MODELS,
    best_fit_path=best_fit_file_2P,
    new_file=True
) 